Nous allons dans ce cours synthétiser ce que nous avons vu jusqu'à présent. Pour cela, nous allons construire une classe intégré dans un module qui permet de faire de la régression linéaire, que vous pouvez réutiliser et/ou partager de façon transparente.

Avant toute chose, **créez un dossier POO** dans le répertoire de votre choix.

<h2> 1ère étape : créer un module permettant de générer nos données. </h2>

$\Rightarrow$ créez le fichier **`simu_data.py`** en y incluant la fonction `linear` que nous avons vue et utilisée dans les cours précédents. Dans ce fichier, pensez à également définir la fonction `main()`. </br>
Remarque : pour plus de clareté pour le code, dorénavent nous appelons cette fonction **`generate_data`** :


In [1]:
import numpy as np

def generate_data(N=100, params=(0,1), var=1):
    """ Fonction simulant les donnees : on genere une distribution de points au
        voisinage d'une droite avec la fonction lineaire : f(x)=a*x+b+N(0,1)
        (cf la fonction linear de Regression_1.ipynb)
        
        N      : nombre de points pour generer le vecteur x
        params : parametres de la fonction : b=params[0] et a=params[1]
        var    : facteur faisant varier la variance de la fonction normale N(0,1)
    """
    x = 10*np.random.random(N)
    y = params[0] + params[1]*x + var*np.random.normal(size=len(x))
    return x, y

En utilisant la fonction `main()` à la fin de notre fichier `sime_data.py`, on peut décider de sauvegarder les points de données générés dans un fichier texte au format csv, pour pouvoir les réutiliser plus tard sans avoir à les générer à chaque fois.

In [2]:
# Partie du script qui sera execute seulement lorsqu'on lance le script directement (python generate_data.py), 
# mais qui ne sera pas execute si on l'importe dans un autre script (import generate_data).
# Remarque : np.savetxt permet de sauvegarder les donnees dans un fichier texte au format .csv .
def main():
    x,y = generate_data()
    np.savetxt("generated.csv", np.column_stack((x,y))) 


if __name__ == '__main__':
    main()

Vous avez maintenant votre propre module `simu_data.py` qui vous permet de générer les données de manière indépendante à leur utilisation.

<h2> Utilisation de ce module </h2>

Pour utiliser ce script en tant que module, créez un fichier **`run.py`** :

In [3]:
from simu_data import generate_data
import matplotlib.pyplot as plt

def main():
    x,y = generate_data(1000, var=5)
    plt.scatter(x, y)
    plt.show()

if __name__ == '__main__':
    main()

ImportError: No module named simu_data

<h2> 2ème étape : création d'un module regroupant les fonctions de la régression linéaire </h2>

Nous allons maintenant **créer un nouveau dossier** que vous allez appeler **`regression`**. Dans ce dossier, nous allons définir un nouveau module en créant le fichier **̀`linear_regression.py`**

**Important !** Pensez à créer le fichier (vide) **`__init__.py`**.

Dans le fichier `linear_regression.py`, nous allons créer une classe **`Linear()`** et définir dans cette classe les différentes fonctions que nous avons utilisé dans le cours "Regression_2_Moindre_Carre_Optimisation".

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


class Linear():
    def __init__(self, x, y):
        if (type(x) is not np.ndarray) and (type(y) is not np.ndarray):
            raise ValueError("Inputs must be numpy arrays")
        self.x_local = x 
        self.y_local = y
    
    def linear_hypothesis(self, x, theta=(0,1)):
        """ Linear function f(x)=a*x+b
        
        Args:
            x (numpy.array()) : vector used to generate the output
            theta (tuple of size 2) : b=params[0] and a=params[1]
        
        Returns:
            numpy.array()
        """
        a, b = theta[1], theta[0]
        return b+a*x
    
    def cost(self, x, y, func, func_args):
        """ Cost function associated with (x,y) couple and function to fit
        
        Args:
            x : feature vector 
            y : explained variable
            func : target function
            func_args : arguments of function
        
        Returns:
            scalar value of cost
        """
        return 1./len(x)*sum(np.square(y-func(x,func_args)))
    
    def params_search(self, x, y):
        """ Estimation des parametres : scan systematique.
        
        Args : x : feature vector 
               y : explained variable
        
        Returns:
            Parametres optimaux de la fonction linear_hypothesis
        """
        errors = []
        params = []
        for p0 in np.linspace(0,10):
            for p1 in np.linspace(0,10):
                p = (p0,p1)
                c = self.cost(x, y, self.linear_hypothesis, p)
                errors.append(c)
                params.append(p)
        return params[errors.index(min(errors))]
    
    def gradient_descent(self, x, y, theta=None, alpha=0.001, iterations=10000):
        """ Estimation des parametres : descente de gradient.
        
        Args : x : feature vector 
               y : explained variable
               theta : starting parameters
               alpha : step
               iterations : iteration number
        
        Returns:
            Parametres optimaux de la fonction linear_hypothesis
        """
        if theta is None:
            theta = (5,10) 

        theta = list(theta)
        N=len(y)
        
        for i in range(iterations):
            theta[0] = theta[0]-(alpha/N)*sum(self.linear_hypothesis(x,theta)-y)
            theta[1] = theta[1]-(alpha/N)*(self.linear_hypothesis(x,theta)-y).dot(x)

        return theta


Maintenant que l'on a regroupé les fonction, on veut les utiliser pour ajuster (*to fit* en anglais) les paramètres aux données. Toujours dans la classe, définissons maintenant la fonction suivante :

In [ ]:
    def fit(self, optimisation=None):
        """ Fonction retournant les parametres estimes a l'aide de la methode des moindres carres.
            
            Args: 
                optimisation : doit etre scan_systematique ou descente_gradient
        """
        if optimisation == "scan_systematique":
            return self.params_search(self.x_local, self.y_local)
        elif optimisation == "descente_gradient":
            return self.gradient_descent(self.x_local, self.y_local, (5,10), alpha=0.01, iterations=5000)
        else:
            raise ValueError("The parameter optimisation must be 'scan_systematique' or 'descente_gradient'")

Tant qu'à faire, on peut également en profiter pour créer 2 fonctions permettant pour l'une d'afficher les données, et pour l'autre d'afficher la droite f(x)=a\*x+b à partir de `self.linear_hypothesis(self.x_local, parameters)`.

In [ ]:
    def plot_data(self, show=False):
        """ Graphe : points de donnees x en fonction de y. 
            
        Args :
            show : affichage optionnel
        """
        plt.scatter(self.x_local,self.y_local)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('')
        if show : 
            plt.show()

    def plot_fit(self, parameters, legende, color='b', show=False):
        """ Graphe : droite de la fonction lineaire f(x)=a*x+b .
        
        Args :
            parameters : parametres de la droite (b=params[0] and a=params[1])
            legende : donne une legende a la droite
            color : change la couleur de la droite => 'b'         blue
                                                      'g'         green
                                                      'r'         red
                                                      'c'         cyan
                                                      'm'         magenta
                                                      'y'         yellow
                                                      'k'         black
                                                      'w'         white
            show : affichage optionnel
        """
        plt.plot(self.x_local, self.linear_hypothesis(self.x_local, parameters), color, label=legende)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.legend()
        plt.title('')
        if show : 
            plt.show()

<h2> 3ème étape : utiliser ces nouveaux modules </h2>

Maintenant que nous avons défini nos nouveaux modules à l'aide de fonctions et de classes, nous devons les utiliser. Remontez dans le **dossier `POO/`** et ouvrez à nouveau le fichier **`run.py`**. 

On peut maintenant importer notre nouveau module **`regression`** et son sous-module **`linear_regression`** de telle manière : 

In [ ]:
from regression import linear_regression

Remarque : on pourrait également importer directement la classe `Linear` d'une de ces façons :

`from regression import linear_regression.Linear as Linear` <br>
ou <br>
`from regression.linear_regression import Linear`

Aide pour écrire le code dans `run.py` :

1. Déclarez la classe `Linear` (comme vu dans Introduction_Python_2).
1. Utilisez ses fonctions afin de déterminer les meilleurs paramètres ajustant les données avec l'option "scan_systematique" puis avec l'option "descente_gradient" de la fonction `fit`.
1. Affichez sur un plot les points de données (fonction `plot_data()` de la classe `Linear`) ainsi que 2 droites avec comme coefficients les paramètres trouvés au point 2. (fonction `plot_fit()` de la classe `Linear`)

In [ ]:
#import matplotlib.pyplot as plt
from simu_data import generate_data
from regression import linear_regression

# Autres manières d'importer directement la classe Linear :
#from regression import linear_regression.Linear as Linear
# ou
#from regression.linear_regression import Linear

def main():
    x,y = generate_data(1000, var=5)
    
    l = linear_regression.Linear(x,y)
    l.plot_data()
    params_scan = l.fit(optimisation="scan_systematique")
    print "Parametres optimisant les moindres carres, via un scan :", params_scan
    l.plot_fit(params_scan, legende="Params scan systematique", color="g")
    
    params_gd = l.fit(optimisation="descente_gradient")
    print "Parametres optimisant les moindres carres, via la descente de gradient :", params_gd
    l.plot_fit(params_gd, legende="Params descente de gradient", color="r", show=True)
    
    # Autre maniere d'utiliser la classe :
    #plt.plot(l.x_local, l.linear_hypothesis(l.x_local, theta = params_gd))
    #plt.show()

if __name__ == '__main__':
    main()